# Graph Neural Networks

# PyTorch

Import:

In [1]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch
from torch_geometric.data import Data
import torch_geometric.transforms as T
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
import torchvision.transforms as transforms
from torch_geometric.nn import GCNConv, GATConv
import torch.nn.functional as F 
from sklearn.manifold import TSNE
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import pandas as pd
import seaborn as sns
import numpy as np
import pickle
import os

In [2]:
INTERMEDIATE_PATH = 'intermediate'
OUTPUT_PATH = 'output\\CY'
DATA_PATH = 'data'

In [3]:
historic_path = os.path.join(INTERMEDIATE_PATH, 'Input_HistoricData_CY.csv')

In [4]:
historic_path

'intermediate\\Input_HistoricData_CY.csv'

In [5]:
if os.path.exists(historic_path):
    historic_data = pd.read_csv(historic_path, sep=',')
    print("Historic data loaded successfully.")

Historic data loaded successfully.


In [6]:
historic_data = historic_data.drop('Unnamed: 0', axis=1)

In [7]:
historic_data.head(1)

,COD_PERIODO,COD_PAIS,TARGET,ALCANCE,ESTRUCTURA_VENTA,COD_SUBESTRATEGIA,DES_SUBESTRATEGIA,COD_CUC,COD_UNIDAD_NEGOCIO,DES_UNIDAD_NEGOCIO,...,PRECIO_OFERTA,FACTOR_REPETICION,REAL_VENTA_NETA_MONEDA_LOCAL,OFFER_UNIT_DSCT,OFFER_DSCT,PUP,DESLINEA,DESSUBCATEGORIA,DESESTATUS,ANIOCAMPANAINTRO
0,202308,PE,CONSULTORA,MASIVO,3384_3541_3392_3392,1,INDIVIDUAL,P0596048576,20,ACCESORIOS,...,29.9,1.0,64946.36,0.660227,0.660227,0.018725,NaN,NaN,NaN,NaN


In [8]:
#OnehotEncoding para campaña intro:
historic_data['ES_CAMPANAINTRO'] = np.where(historic_data['COD_PERIODO']==historic_data['ANIOCAMPANAINTRO'],1,0)

In [9]:
#Eliminar columnas 
historic_data = historic_data.drop(['TARGET','COD_PAIS','ALCANCE','ANIOCAMPANAINTRO','COD_SUBESTRATEGIA',
       'DES_SUBESTRATEGIA','DESESTATUS','COD_MARCA', 'DES_MARCA'], axis=1)

In [10]:
historic_data.shape

(7706, 23)

In [11]:
historic_data.columns

Index(['COD_PERIODO', 'ESTRUCTURA_VENTA', 'COD_CUC', 'COD_UNIDAD_NEGOCIO',
       'DES_UNIDAD_NEGOCIO', 'DES_CLASE', 'COD_CATEGORIA', 'DES_CATEGORIA',
       'COD_TIPOSOLO', 'DES_TIPOSOLO', 'ES_EMPAQUE', 'ES_MUESTRA', 'DES_CUC',
       'PRECIO_NORMAL', 'PRECIO_OFERTA', 'FACTOR_REPETICION',
       'REAL_VENTA_NETA_MONEDA_LOCAL', 'OFFER_UNIT_DSCT', 'OFFER_DSCT', 'PUP',
       'DESLINEA', 'DESSUBCATEGORIA', 'ES_CAMPANAINTRO'],
      dtype='object')

In [12]:
data = historic_data.copy()

In [13]:
#Eliminar columnas 
data = data.drop(['DES_CUC','DES_UNIDAD_NEGOCIO','DES_CATEGORIA','DES_TIPOSOLO','FACTOR_REPETICION'], axis=1)

In [14]:
# Codificación de atributos categóricos 'DES_CATEGORIA' y 'DES_TIPOSOLO'
# Rellenar valores nulos con una cadena especial antes de codificar
data['COD_UNIDAD_NEGOCIO'].fillna('UNKNOWN', inplace=True)
data['DES_CLASE'].fillna('UNKNOWN', inplace=True)
data['COD_CATEGORIA'].fillna('UNKNOWN', inplace=True)
data['COD_TIPOSOLO'].fillna('UNKNOWN', inplace=True)
data['DESLINEA'].fillna('UNKNOWN', inplace=True)
data['DESSUBCATEGORIA'].fillna('UNKNOWN', inplace=True)

# Crear codificadores LabelEncoder para transformar los valores categóricos
le_unidad = LabelEncoder()
le_clase = LabelEncoder()
le_categoria = LabelEncoder()
le_tiposolo = LabelEncoder()
le_deslinea = LabelEncoder()
le_subcategoria = LabelEncoder()

# Aplicar codificación y convertir a tensores
data['COD_UNIDAD_NEGOCIO_encoded'] = le_categoria.fit_transform(data['COD_UNIDAD_NEGOCIO'])
data['DES_CLASE_encoded'] = le_tiposolo.fit_transform(data['DES_CLASE'])
data['DES_CATEGORIA_encoded'] = le_categoria.fit_transform(data['COD_CATEGORIA'])
data['DES_TIPOSOLO_encoded'] = le_tiposolo.fit_transform(data['COD_TIPOSOLO'])
data['DES_LINEA_encoded'] = le_deslinea.fit_transform(data['DESLINEA'])
data['DES_SUBCATEGORIA_encoded'] = le_subcategoria.fit_transform(data['DESSUBCATEGORIA'])

C:\Users\sheil\AppData\Local\Temp\ipykernel_42940\1551182746.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['COD_UNIDAD_NEGOCIO'].fillna('UNKNOWN', inplace=True)
C:\Users\sheil\AppData\Local\Temp\ipykernel_42940\1551182746.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

In [15]:
data = data[['COD_PERIODO','ESTRUCTURA_VENTA','COD_CUC', 
             'ES_EMPAQUE','ES_MUESTRA','PRECIO_NORMAL','PRECIO_OFERTA',
             'REAL_VENTA_NETA_MONEDA_LOCAL', 'OFFER_UNIT_DSCT', 'OFFER_DSCT', 'PUP',
             'ES_CAMPANAINTRO',
             'COD_UNIDAD_NEGOCIO_encoded', 'DES_CLASE_encoded',
             'DES_CATEGORIA_encoded', 'DES_TIPOSOLO_encoded', 'DES_LINEA_encoded',
             'DES_SUBCATEGORIA_encoded']].copy()

In [16]:
data['ES_EMPAQUE'].fillna(0, inplace=True)
data['ES_MUESTRA'].fillna(0, inplace=True)
data['PRECIO_NORMAL'].fillna(0, inplace=True)

C:\Users\sheil\AppData\Local\Temp\ipykernel_42940\3760813815.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['ES_EMPAQUE'].fillna(0, inplace=True)
C:\Users\sheil\AppData\Local\Temp\ipykernel_42940\3760813815.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

In [17]:
data.shape

(7706, 18)

In [18]:
import logging
# Construcción de grafos por campaña
def construir_grafos(data, output_path=OUTPUT_PATH):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    grafos_por_campaña = {}
    for campaña, datos_campaña in data.groupby('COD_PERIODO'):
        logging.info(f"Generando el grafo para la campaña {campaña}")
        G = nx.Graph()

        # Agregar nodos con atributos
        for _, row in datos_campaña.iterrows():
            nodo_id = row['COD_CUC']
            atributos_nodo = {
                'ES_EMPAQUE': row['ES_EMPAQUE'],
                'ES_MUESTRA': row['ES_MUESTRA'],
                'PRECIO_NORMAL': row['PRECIO_NORMAL'],
                'DES_CATEGORIA_encoded': row['DES_CATEGORIA_encoded'],
                'DES_TIPOSOLO_encoded': row['DES_TIPOSOLO_encoded'],
                'DES_LINEA_encoded': row['DES_LINEA_encoded'],
                'DES_SUBCATEGORIA_encoded': row['DES_SUBCATEGORIA_encoded'],
                'ES_CAMPANAINTRO': row['ES_CAMPANAINTRO']
            }
            G.add_node(nodo_id, **atributos_nodo)

        # Crear aristas si están en la misma estructura de ventas
        estructura_grupos = datos_campaña.groupby('ESTRUCTURA_VENTA')
        for _, grupo in estructura_grupos:
            nodos = grupo['COD_CUC'].tolist()
            for i in range(len(nodos)):
                for j in range(i + 1, len(nodos)):
                    G.add_edge(nodos[i], nodos[j])

        # Guardar el grafo generado
        file_path = os.path.join(output_path, f"grafo_campaña_{campaña}.pkl")
        with open(file_path, "wb") as f:
            pickle.dump(G, f)
        print(f"Grafo de la campaña {campaña} guardado en {file_path}")

        grafos_por_campaña[campaña] = G

    return grafos_por_campaña

In [19]:
# Ejecutar la función con tu dataframe `data`
grafos_por_campaña = construir_grafos(data)

Grafo de la campaña 202204 guardado en output\CY\grafo_campaña_202204.pkl
Grafo de la campaña 202205 guardado en output\CY\grafo_campaña_202205.pkl
Grafo de la campaña 202206 guardado en output\CY\grafo_campaña_202206.pkl
Grafo de la campaña 202207 guardado en output\CY\grafo_campaña_202207.pkl
Grafo de la campaña 202208 guardado en output\CY\grafo_campaña_202208.pkl
Grafo de la campaña 202209 guardado en output\CY\grafo_campaña_202209.pkl
Grafo de la campaña 202210 guardado en output\CY\grafo_campaña_202210.pkl
Grafo de la campaña 202211 guardado en output\CY\grafo_campaña_202211.pkl
Grafo de la campaña 202212 guardado en output\CY\grafo_campaña_202212.pkl
Grafo de la campaña 202213 guardado en output\CY\grafo_campaña_202213.pkl
Grafo de la campaña 202214 guardado en output\CY\grafo_campaña_202214.pkl
Grafo de la campaña 202215 guardado en output\CY\grafo_campaña_202215.pkl
Grafo de la campaña 202216 guardado en output\CY\grafo_campaña_202216.pkl
Grafo de la campaña 202217 guardado en

In [20]:
G = grafos_por_campaña[202404] 
plt.figure(figsize=(150, 150))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_size=20, node_color="skyblue", font_size=10, font_weight="bold", edge_color="gray")
edge_labels = nx.get_edge_attributes(G, 'tipo')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color="red")
plt.title(f"Grafo de relaciones de productos para la campaña {202404}")
plt.show()

In [21]:
import pandas as pd
import torch
import numpy as np
from torch_geometric.nn import SAGEConv
from sklearn.manifold import TSNE
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.utils import from_networkx
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
import random
import os
import pickle
import json
import logging

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Modelo GraphSAGE con regularización L2 (mejorado)
class GraphSAGEModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGEModel, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, out_channels)
        self.dropout = 0.5
        self.reg_lambda = 1e-5  # Regularización L2

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv3(x, edge_index)
        return x

    def regularization_loss(self):
        reg_loss = 0
        for param in self.parameters():
            reg_loss += self.reg_lambda * param.norm(2)
        return reg_loss


# Modelo de predicción de enlaces
class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels):
        super(LinkPredictor, self).__init__()
        self.lin1 = torch.nn.Linear(in_channels * 2, 64)
        self.lin2 = torch.nn.Linear(64, 32)
        self.lin3 = torch.nn.Linear(32, 1)

    def forward(self, x_i, x_j):
        x = torch.cat([x_i, x_j], dim=-1)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        return torch.sigmoid(self.lin3(x))


# Visualización de embeddings usando t-SNE
def graficar_embeddings(embeddings, campaña, output_path):
    tsne = TSNE(n_components=2, random_state=42)
    embeddings_2d = tsne.fit_transform(embeddings.cpu().numpy())
    # Guardar visualización
    plt.figure(figsize=(10, 7))
    plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.7, cmap='viridis')
    plt.colorbar(label="Proyección t-SNE")
    plt.title(f"Embeddings visualizados con t-SNE (Campaña {campaña})")
    plt.savefig(os.path.join(output_path, f"embeddings_campaña_{campaña}.png"))
    plt.close()

    # Guardar embeddings como tensor
    embeddings_file = os.path.join(output_path, f"embeddings_campaña_{campaña}.pt")
    torch.save(embeddings, embeddings_file)
    print(f"Embeddings guardados en {embeddings_file}")


# Generar aristas negativas
def generar_aristas_negativas(data_pg, num_negativas):
    all_nodes = torch.arange(data_pg.x.size(0))  # Todos los nodos del grafo
    existing_edges = set(map(tuple, data_pg.edge_index.t().tolist()))  # Aristas existentes
    neg_edges = set()

    while len(neg_edges) < num_negativas:
        i, j = random.sample(all_nodes.tolist(), 2)
        if (i != j) and ((i, j) not in existing_edges) and ((j, i) not in existing_edges):
            neg_edges.add((i, j))

    neg_edges = torch.tensor(list(neg_edges), dtype=torch.long).t()
    return neg_edges


# GridSearch para buscar los mejores hiperparámetros
def buscar_mejores_hiperparametros(grafos_por_campaña, output_path=OUTPUT_PATH):
    param_grid = {
        'hidden_channels': [32, 64],
        'learning_rate': [0.001, 0.0001],
        'dropout': [0.3, 0.5],
        'reg_lambda': [1e-5, 1e-6]
    }

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    mejores_hiperparametros = {}

    for campaña, G in grafos_por_campaña.items():
        print(f"Buscando mejores hiperparámetros para la campaña {campaña}")
        
        node_mapping = {node: idx for idx, node in enumerate(G.nodes)}
        data_pg = from_networkx(G)
        data_pg.edge_index = torch.tensor([
            [node_mapping[src], node_mapping[dst]] for src, dst in G.edges
        ]).t()

        data_pg.x = torch.tensor([
            [G.nodes[node].get('PRECIO_NORMAL', 0),
             G.nodes[node].get('DES_CATEGORIA_encoded', 0),
             G.nodes[node].get('DES_TIPOSOLO_encoded', 0),
             G.nodes[node].get('DES_LINEA_encoded', 0),
             G.nodes[node].get('DES_SUBCATEGORIA_encoded', 0),
             G.nodes[node].get('ES_EMPAQUE', 0),
             G.nodes[node].get('ES_MUESTRA', 0),
             G.nodes[node].get('ES_CAMPANAINTRO', 0)
             ] for node in G.nodes
        ], dtype=torch.float)

        num_negativas = data_pg.edge_index.size(1)
        neg_edges = generar_aristas_negativas(data_pg, num_negativas)

        all_edges = torch.cat([data_pg.edge_index, neg_edges], dim=1)
        all_labels = torch.cat([
            torch.ones(data_pg.edge_index.size(1)),
            torch.zeros(neg_edges.size(1))
        ])
        train_edges, test_edges, train_labels, test_labels = train_test_split(
            all_edges.t().numpy(), all_labels.numpy(), test_size=0.2, stratify=all_labels.numpy(), random_state=42
        )
        train_edge_index = torch.tensor(train_edges, dtype=torch.long).t()
        train_edge_attr = torch.tensor(train_labels, dtype=torch.float)
        test_edge_index = torch.tensor(test_edges, dtype=torch.long).t()
        test_edge_attr = torch.tensor(test_labels, dtype=torch.float)

        best_params = None
        best_auc = 0

        for params in ParameterGrid(param_grid):
            print(f"Probando hiperparámetros: {params}")
            model = GraphSAGEModel(
                in_channels=data_pg.x.size(1),
                hidden_channels=params['hidden_channels'],
                out_channels=16
            )
            model.dropout = params['dropout']
            model.reg_lambda = params['reg_lambda']

            optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
            model.train()

            for epoch in range(500):  # Fase de búsqueda rápida
                optimizer.zero_grad()
                embeddings = model(data_pg.x, train_edge_index)
                edge_labels = train_edge_attr
                edge_predictions = torch.sigmoid(
                    (embeddings[train_edge_index[0]] * embeddings[train_edge_index[1]]).sum(dim=1)
                )
                loss = F.binary_cross_entropy(edge_predictions, edge_labels) + model.regularization_loss()
                loss.backward()
                optimizer.step()

            model.eval()
            with torch.no_grad():
                embeddings = model(data_pg.x, data_pg.edge_index)
                test_preds = torch.sigmoid(
                    (embeddings[test_edge_index[0]] * embeddings[test_edge_index[1]]).sum(dim=1)
                )
                auc_score = roc_auc_score(test_edge_attr.cpu(), test_preds.cpu())

            if auc_score > best_auc:
                best_auc = auc_score
                best_params = params

        mejores_hiperparametros[campaña] = best_params

    with open(os.path.join(output_path, "mejores_hiperparametros.json"), "w") as f:
        json.dump(mejores_hiperparametros, f, indent=4)
    print(f"Hiperparámetros guardados en {output_path}/mejores_hiperparametros.json")


# Ejecutar GridSearch
buscar_mejores_hiperparametros(grafos_por_campaña, output_path=OUTPUT_PATH)


Buscando mejores hiperparámetros para la campaña 202204
Probando hiperparámetros: {'dropout': 0.3, 'hidden_channels': 32, 'learning_rate': 0.001, 'reg_lambda': 1e-05}
Probando hiperparámetros: {'dropout': 0.3, 'hidden_channels': 32, 'learning_rate': 0.001, 'reg_lambda': 1e-06}
Probando hiperparámetros: {'dropout': 0.3, 'hidden_channels': 32, 'learning_rate': 0.0001, 'reg_lambda': 1e-05}
Probando hiperparámetros: {'dropout': 0.3, 'hidden_channels': 32, 'learning_rate': 0.0001, 'reg_lambda': 1e-06}
Probando hiperparámetros: {'dropout': 0.3, 'hidden_channels': 64, 'learning_rate': 0.001, 'reg_lambda': 1e-05}
Probando hiperparámetros: {'dropout': 0.3, 'hidden_channels': 64, 'learning_rate': 0.001, 'reg_lambda': 1e-06}
Probando hiperparámetros: {'dropout': 0.3, 'hidden_channels': 64, 'learning_rate': 0.0001, 'reg_lambda': 1e-05}
Probando hiperparámetros: {'dropout': 0.3, 'hidden_channels': 64, 'learning_rate': 0.0001, 'reg_lambda': 1e-06}
Probando hiperparámetros: {'dropout': 0.5, 'hidden_c

In [22]:
def graficar_analisis_significatividad(importancia_permutacion, importancia_gradiente, correlaciones, campaña, output_path):
    """
    Genera gráficos para visualizar el análisis de significatividad de características con una paleta personalizada.
    Args:
        importancia_permutacion (dict): Importancia de las características por permutación.
        importancia_gradiente (tensor): Importancia de las características por gradiente.
        correlaciones (list): Correlaciones entre características y embeddings.
        campaña (int): Identificador de la campaña.
        output_path (str): Carpeta donde guardar los gráficos.
    """
    # Paleta de colores personalizada
    colores = ['#2C3E50', '#34495E', '#7F8C8D', '#95A5A6', '#4D5656', '#BDC3C7']

    # Asegurar que el directorio de salida exista
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Gráfico de importancia por permutación
    plt.figure(figsize=(10, 6))
    plt.bar(importancia_permutacion.keys(), importancia_permutacion.values(), color=colores[0])
    plt.title(f"Importancia por Permutación - Campaña {campaña}")
    plt.ylabel("Impacto en AUC")
    plt.xlabel("Características")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(os.path.join(output_path, f"importancia_permutacion_campaña_{campaña}.png"))
    plt.close()

    # Gráfico de importancia por gradiente
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(importancia_gradiente)), importancia_gradiente.numpy(), color=colores[1])
    plt.title(f"Importancia por Gradiente - Campaña {campaña}")
    plt.ylabel("Magnitud del Gradiente")
    plt.xlabel("Características")
    plt.xticks(range(len(importancia_gradiente)), [f"Feature_{i}" for i in range(len(importancia_gradiente))], rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(os.path.join(output_path, f"importancia_gradiente_campaña_{campaña}.png"))
    plt.close()

    # Gráfico de correlaciones con embeddings
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(correlaciones)), correlaciones, color=colores[2])
    plt.title(f"Correlaciones con Embeddings - Campaña {campaña}")
    plt.ylabel("Coeficiente de Correlación")
    plt.xlabel("Características")
    plt.xticks(range(len(correlaciones)), [f"Feature_{i}" for i in range(len(correlaciones))], rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(os.path.join(output_path, f"correlaciones_embeddings_campaña_{campaña}.png"))
    plt.close()


In [23]:
# Evaluar modelo
def evaluar_modelo(link_predictor, test_edge_index, test_edge_attr, embeddings):
    with torch.no_grad():
        preds = link_predictor(
            embeddings[test_edge_index[0]], embeddings[test_edge_index[1]]
        ).squeeze()
        auc_score = roc_auc_score(test_edge_attr.cpu(), preds.cpu())
    return auc_score


# **1. Permutación de Features**
def calcular_importancia_permutacion(model, link_predictor, data_pg, test_edge_index, test_edge_attr, embeddings):
    baseline_auc = evaluar_modelo(link_predictor, test_edge_index, test_edge_attr, embeddings)
    importancias = {}

    for col in range(data_pg.x.size(1)):
        x_permutado = data_pg.x.clone()
        x_permutado[:, col] = x_permutado[torch.randperm(data_pg.x.size(0)), col]
        
        embeddings_permutados = model(x_permutado, data_pg.edge_index)
        auc_permutado = evaluar_modelo(link_predictor, test_edge_index, test_edge_attr, embeddings_permutados)
        importancias[f"Feature_{col}"] = baseline_auc - auc_permutado

    return importancias


# **2. Gradiente de Entrada**
def calcular_gradiente_importancia(model, data_pg):
    data_pg.x.requires_grad = True
    embeddings = model(data_pg.x, data_pg.edge_index)

    importancia_gradiente = torch.zeros(data_pg.x.size(1))
    for i in range(data_pg.x.size(0)):
        embedding = embeddings[i]
        embedding.mean().backward(retain_graph=True)
        importancia_gradiente += data_pg.x.grad[i].abs()

    return importancia_gradiente / data_pg.x.size(0)


# **3. Correlación con Embeddings**
def calcular_correlacion_embeddings(data_pg, embeddings):
    correlaciones = []
    for col in range(data_pg.x.size(1)):
        # Usar detach() para evitar problemas con gradientes
        col_data = data_pg.x[:, col].detach().cpu().numpy()
        embedding_data = embeddings.cpu().detach().numpy().mean(axis=1)
        corr = np.corrcoef(col_data, embedding_data)[0, 1]
        correlaciones.append(corr)
    return correlaciones


In [24]:
def identificar_aristas_positivas(link_predictor, test_edge_index, embeddings, threshold=0.8):
    with torch.no_grad():
        preds = link_predictor(
            embeddings[test_edge_index[0]], embeddings[test_edge_index[1]]
        ).squeeze()
        aristas_positivas = test_edge_index[:, (preds > threshold)]
    return aristas_positivas


# **2. Guardar resultados de las aristas positivas**
def guardar_aristas_positivas(aristas_positivas, campaña, output_path):
    aristas_positivas_lista = aristas_positivas.cpu().numpy().tolist()
    file_path = os.path.join(output_path, f"aristas_positivas_campaña_{campaña}.json")
    with open(file_path, "w") as f:
        json.dump(aristas_positivas_lista, f, indent=4)
    print(f"Aristas positivas guardadas en {file_path}")

def graficar_grafo_aristas_positivas(G, aristas_positivas, campaña, output_path, reverse_mapping, link_predictions=None):
    """
    Graficar el grafo original resaltando las aristas predichas como positivas.
    Args:
        G (networkx.Graph): Grafo original.
        aristas_positivas (torch.Tensor): Aristas positivas en formato de índices.
        campaña (int): Identificador de la campaña.
        output_path (str): Carpeta de salida para guardar el grafo.
        reverse_mapping (dict): Mapeo de índices de nodos a sus etiquetas originales.
        link_predictions (torch.Tensor, optional): Probabilidades predichas para las aristas.
    """
    plt.figure(figsize=(12, 10))

    # Convertir los índices de las aristas positivas a etiquetas originales
    aristas_positivas_tuples = [
        (reverse_mapping[int(u)], reverse_mapping[int(v)])
        for u, v in aristas_positivas.t().tolist()
        if reverse_mapping[int(u)] in G.nodes and reverse_mapping[int(v)] in G.nodes
    ]

    # Crear un subgrafo solo con los nodos conectados por al menos una arista positiva
    nodos_conectados = set([nodo for edge in aristas_positivas_tuples for nodo in edge])
    subgrafo = G.subgraph(nodos_conectados).copy()

    # Generar posiciones para los nodos
    pos = nx.spring_layout(subgrafo, seed=42)

    # Dibujar nodos del subgrafo
    nx.draw(subgrafo, pos, with_labels=True, node_color="lightblue", edge_color="gray", alpha=0.7, node_size=500)

    # Asignar colores a las aristas positivas
    if link_predictions is not None:
        # Normalizar las probabilidades para usarlas como gradiente de color
        probs = link_predictions.cpu().detach().numpy()
        nx.draw_networkx_edges(
            subgrafo,
            pos,
            edgelist=aristas_positivas_tuples,
            edge_color=probs,
            edge_cmap=plt.cm.Reds,  # Usar gradiente de colores
            width=2.0,
            alpha=0.8,
        )
    else:
        # Usar rojo como predeterminado si no se especifican probabilidades
        nx.draw_networkx_edges(
            subgrafo,
            pos,
            edgelist=aristas_positivas_tuples,
            edge_color="gray",
            width=2.0,
            alpha=0.8,
        )

    plt.title(f"Grafo con Aristas Positivas - Campaña {campaña}")
    plt.savefig(os.path.join(output_path, f"grafo_aristas_positivas_campaña_{campaña}.png"))
    plt.close()
    print(f"Grafo con aristas positivas guardado en {os.path.join(output_path, f'grafo_aristas_positivas_campaña_{campaña}.png')}")

In [25]:

# Entrenamiento y predicción
def entrenar_y_predecir(grafos_por_campaña, guardar_embeddings=False, output_path=OUTPUT_PATH):
    # Cargar los mejores hiperparámetros encontrados
    with open(os.path.join(output_path, "mejores_hiperparametros.json"), "r") as f:
        mejores_hiperparametros = json.load(f)

    resultados_por_campaña = {}

    for campaña, params in mejores_hiperparametros.items():
        print(f"Entrenando para la campaña {campaña} con parámetros: {params}")
        
        campaña = int(campaña)

        # Obtener el grafo de la campaña
        G = grafos_por_campaña[campaña]

        # Mapear nodos a índices
        node_mapping = {node: idx for idx, node in enumerate(G.nodes)}
        data_pg = from_networkx(G)
        data_pg.edge_index = torch.tensor([
            [node_mapping[src], node_mapping[dst]] for src, dst in G.edges
        ]).t()

        # Generar atributos de nodo
        data_pg.x = torch.tensor([
            [G.nodes[node].get('PRECIO_NORMAL', 0),
             G.nodes[node].get('DES_CATEGORIA_encoded', 0),
             G.nodes[node].get('DES_TIPOSOLO_encoded', 0),
             G.nodes[node].get('DES_LINEA_encoded', 0),
             G.nodes[node].get('DES_SUBCATEGORIA_encoded', 0),
             G.nodes[node].get('ES_EMPAQUE', 0),
             G.nodes[node].get('ES_MUESTRA', 0),
             G.nodes[node].get('ES_CAMPANAINTRO', 0)
             ] for node in G.nodes
        ], dtype=torch.float)

        # Generar aristas negativas
        num_negativas = data_pg.edge_index.size(1)
        neg_edges = generar_aristas_negativas(data_pg, num_negativas)

        # Crear dataset de entrenamiento y prueba
        all_edges = torch.cat([data_pg.edge_index, neg_edges], dim=1)
        all_labels = torch.cat([
            torch.ones(data_pg.edge_index.size(1)),
            torch.zeros(neg_edges.size(1))
        ])
        train_edges, test_edges, train_labels, test_labels = train_test_split(
            all_edges.t().numpy(), all_labels.numpy(), test_size=0.2, stratify=all_labels.numpy(), random_state=42
        )
        train_edge_index = torch.tensor(train_edges, dtype=torch.long).t()
        train_edge_attr = torch.tensor(train_labels, dtype=torch.float)
        test_edge_index = torch.tensor(test_edges, dtype=torch.long).t()
        test_edge_attr = torch.tensor(test_labels, dtype=torch.float)

        # Usar los mejores hiperparámetros para el modelo
        model = GraphSAGEModel(
            in_channels=data_pg.x.size(1),
            hidden_channels=params['hidden_channels'],
            out_channels=16
        )
        model.dropout = params['dropout']
        model.reg_lambda = params['reg_lambda']

        optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
        model.train()

        # Entrenar GraphSAGE
        for epoch in range(3000):  # Entrenamiento completo
            optimizer.zero_grad()
            embeddings = model(data_pg.x, train_edge_index)
            edge_labels = train_edge_attr
            edge_predictions = torch.sigmoid(
                (embeddings[train_edge_index[0]] * embeddings[train_edge_index[1]]).sum(dim=1)
            )
            loss = F.binary_cross_entropy(edge_predictions, edge_labels) + model.regularization_loss()
            loss.backward()
            optimizer.step()

            if epoch % 50 == 0:
                print(f"Campaña {campaña}, Epoch {epoch}, Pérdida: {loss.item():.4f}")

        # Generar embeddings
        model.eval()
        with torch.no_grad():
            embeddings = model(data_pg.x, data_pg.edge_index)

        # Guardar y visualizar embeddings si está habilitado
        if guardar_embeddings:
            graficar_embeddings(embeddings, campaña, output_path)

        # Evaluación final
        link_predictor = LinkPredictor(in_channels=16)
        optimizer_link = torch.optim.Adam(link_predictor.parameters(), lr=params['learning_rate'])
        link_predictor.train()

        for epoch in range(3000):
            optimizer_link.zero_grad()
            preds = link_predictor(
                embeddings[test_edge_index[0]], embeddings[test_edge_index[1]]
            ).squeeze()
            loss = F.binary_cross_entropy(preds, test_edge_attr)
            loss.backward()
            optimizer_link.step()

            if epoch % 50 == 0:
                print(f"Campaña {campaña}, Epoch {epoch}, Pérdida del predictor: {loss.item():.4f}")

        # Evaluación de métricas
        link_predictor.eval()
        with torch.no_grad():
            test_preds = link_predictor(
                embeddings[test_edge_index[0]], embeddings[test_edge_index[1]]
            ).squeeze()
            auc_score = roc_auc_score(test_edge_attr.cpu(), test_preds.cpu())
            test_preds_binary = (test_preds > 0.5).float()
            precision = precision_score(test_edge_attr.cpu(), test_preds_binary.cpu())
            recall = recall_score(test_edge_attr.cpu(), test_preds_binary.cpu())
            f1 = f1_score(test_edge_attr.cpu(), test_preds_binary.cpu())

            print(f"Campaña {campaña}: AUC = {auc_score:.4f}, Precision = {precision:.4f}, Recall = {recall:.4f}, F1 = {f1:.4f}")
            resultados_por_campaña[campaña] = {
                "AUC": auc_score,
                "Precision": precision,
                "Recall": recall,
                "F1": f1
            }

        # **Análisis de significatividad**
        print(f"--- Análisis de significatividad para campaña {campaña} ---")
        importancia_permutacion = calcular_importancia_permutacion(
            model, link_predictor, data_pg, test_edge_index, test_edge_attr, embeddings
        )
        importancia_gradiente = calcular_gradiente_importancia(model, data_pg)
        correlaciones = calcular_correlacion_embeddings(data_pg, embeddings)

        print(f"Importancia por permutación: {importancia_permutacion}")
        print(f"Importancia por gradiente: {importancia_gradiente}")
        print(f"Correlaciones con embeddings: {correlaciones}")

        # Guardar métricas y análisis
        resultados_por_campaña[campaña].update({
            "Importancia_Permutacion": importancia_permutacion,
            "Importancia_Gradiente": importancia_gradiente.tolist(),
            "Correlaciones_Embeddings": correlaciones
        })

        graficar_analisis_significatividad(
            importancia_permutacion,
            importancia_gradiente,
            correlaciones,
            campaña,
            output_path
            )
        
        #reverse_mapping = {idx: node for node, idx in node_mapping.items()}
        # Identificar, guardar y graficar aristas positivas
        # Crear el mapeo inverso
        reverse_mapping = {idx: node for node, idx in node_mapping.items()}

        # Identificar, guardar y graficar aristas positivas
        aristas_positivas = identificar_aristas_positivas(link_predictor, test_edge_index, embeddings)
        guardar_aristas_positivas(aristas_positivas, campaña, output_path)
        graficar_grafo_aristas_positivas(G, aristas_positivas, campaña, output_path, reverse_mapping)

    # Guardar resultados
    with open(os.path.join(output_path, "resultados_por_campaña.json"), "w") as f:
        json.dump(resultados_por_campaña, f, indent=4)

    print(f"Resultados guardados en {os.path.join(output_path, 'resultados_por_campaña.json')}")

# Entrenar y predecir usando los mejores hiperparámetros
entrenar_y_predecir(grafos_por_campaña, guardar_embeddings=True)

Entrenando para la campaña 202204 con parámetros: {'dropout': 0.5, 'hidden_channels': 64, 'learning_rate': 0.001, 'reg_lambda': 1e-05}
Campaña 202204, Epoch 0, Pérdida: 45.9956
Campaña 202204, Epoch 50, Pérdida: 49.1740
Campaña 202204, Epoch 100, Pérdida: 48.4538
Campaña 202204, Epoch 150, Pérdida: 48.4538
Campaña 202204, Epoch 200, Pérdida: 47.4229
Campaña 202204, Epoch 250, Pérdida: 45.3610
Campaña 202204, Epoch 300, Pérdida: 48.4538
Campaña 202204, Epoch 350, Pérdida: 47.4229
Campaña 202204, Epoch 400, Pérdida: 44.4873
Campaña 202204, Epoch 450, Pérdida: 48.4538
Campaña 202204, Epoch 500, Pérdida: 49.4848
Campaña 202204, Epoch 550, Pérdida: 49.4848
Campaña 202204, Epoch 600, Pérdida: 49.4848
Campaña 202204, Epoch 650, Pérdida: 49.4848
Campaña 202204, Epoch 700, Pérdida: 49.4848
Campaña 202204, Epoch 750, Pérdida: 49.4848
Campaña 202204, Epoch 800, Pérdida: 47.4229
Campaña 202204, Epoch 850, Pérdida: 47.8602
Campaña 202204, Epoch 900, Pérdida: 46.3920
Campaña 202204, Epoch 950, Pérdi

C:\Users\sheil\AppData\Local\Temp\ipykernel_42940\2261630590.py:68: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.7, cmap='viridis')


Embeddings guardados en output\CY\embeddings_campaña_202204.pt
Campaña 202204, Epoch 0, Pérdida del predictor: 1.4973
Campaña 202204, Epoch 50, Pérdida del predictor: 0.0994
Campaña 202204, Epoch 100, Pérdida del predictor: 0.0233
Campaña 202204, Epoch 150, Pérdida del predictor: 0.0077
Campaña 202204, Epoch 200, Pérdida del predictor: 0.0037
Campaña 202204, Epoch 250, Pérdida del predictor: 0.0022
Campaña 202204, Epoch 300, Pérdida del predictor: 0.0014
Campaña 202204, Epoch 350, Pérdida del predictor: 0.0010
Campaña 202204, Epoch 400, Pérdida del predictor: 0.0008
Campaña 202204, Epoch 450, Pérdida del predictor: 0.0006
Campaña 202204, Epoch 500, Pérdida del predictor: 0.0005
Campaña 202204, Epoch 550, Pérdida del predictor: 0.0004
Campaña 202204, Epoch 600, Pérdida del predictor: 0.0003
Campaña 202204, Epoch 650, Pérdida del predictor: 0.0003
Campaña 202204, Epoch 700, Pérdida del predictor: 0.0002
Campaña 202204, Epoch 750, Pérdida del predictor: 0.0002
Campaña 202204, Epoch 800, P

c:\Users\sheil\OneDrive\Desktop\Proyecto_Tesis\tesis\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\sheil\OneDrive\Desktop\Proyecto_Tesis\tesis\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Aristas positivas guardadas en output\CY\aristas_positivas_campaña_202204.json
Grafo con aristas positivas guardado en output\CY\grafo_aristas_positivas_campaña_202204.png
Entrenando para la campaña 202205 con parámetros: {'dropout': 0.3, 'hidden_channels': 32, 'learning_rate': 0.001, 'reg_lambda': 1e-06}
Campaña 202205, Epoch 0, Pérdida: 44.8718
Campaña 202205, Epoch 50, Pérdida: 43.3988
Campaña 202205, Epoch 100, Pérdida: 39.7437
Campaña 202205, Epoch 150, Pérdida: 35.8974
Campaña 202205, Epoch 200, Pérdida: 36.0341
Campaña 202205, Epoch 250, Pérdida: 37.1795
Campaña 202205, Epoch 300, Pérdida: 32.0517
Campaña 202205, Epoch 350, Pérdida: 34.6155
Campaña 202205, Epoch 400, Pérdida: 41.0257
Campaña 202205, Epoch 450, Pérdida: 38.4616
Campaña 202205, Epoch 500, Pérdida: 38.4616
Campaña 202205, Epoch 550, Pérdida: 39.7436
Campaña 202205, Epoch 600, Pérdida: 38.4616
Campaña 202205, Epoch 650, Pérdida: 37.1795
Campaña 202205, Epoch 700, Pérdida: 38.4616
Campaña 202205, Epoch 750, Pérdida: 

...

Aristas positivas guardadas en output\CY\aristas_positivas_campaña_202403.json
Grafo con aristas positivas guardado en output\CY\grafo_aristas_positivas_campaña_202403.png
Entrenando para la campaña 202404 con parámetros: {'dropout': 0.3, 'hidden_channels': 32, 'learning_rate': 0.001, 'reg_lambda': 1e-06}
Campaña 202404, Epoch 0, Pérdida: 50.0510
Campaña 202404, Epoch 50, Pérdida: 50.0000
Campaña 202404, Epoch 100, Pérdida: 47.7440
Campaña 202404, Epoch 150, Pérdida: 50.0000
Campaña 202404, Epoch 200, Pérdida: 50.0000
Campaña 202404, Epoch 250, Pérdida: 50.0000
Campaña 202404, Epoch 300, Pérdida: 50.0000
Campaña 202404, Epoch 350, Pérdida: 50.0000
Campaña 202404, Epoch 400, Pérdida: 50.0000
Campaña 202404, Epoch 450, Pérdida: 50.0000
Campaña 202404, Epoch 500, Pérdida: 50.0000
Campaña 202404, Epoch 550, Pérdida: 50.0000
Campaña 202404, Epoch 600, Pérdida: 50.0000
Campaña 202404, Epoch 650, Pérdida: 50.0000
Campaña 202404, Epoch 700, Pérdida: 50.0000
Campaña 202404, Epoch 750, Pérdida: 

C:\Users\sheil\AppData\Local\Temp\ipykernel_42940\2261630590.py:68: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.7, cmap='viridis')


Embeddings guardados en output\CY\embeddings_campaña_202404.pt
Campaña 202404, Epoch 0, Pérdida del predictor: 0.6935
Campaña 202404, Epoch 50, Pérdida del predictor: 0.1957
Campaña 202404, Epoch 100, Pérdida del predictor: 0.0577
Campaña 202404, Epoch 150, Pérdida del predictor: 0.0155
Campaña 202404, Epoch 200, Pérdida del predictor: 0.0046
Campaña 202404, Epoch 250, Pérdida del predictor: 0.0018
Campaña 202404, Epoch 300, Pérdida del predictor: 0.0010
Campaña 202404, Epoch 350, Pérdida del predictor: 0.0007
Campaña 202404, Epoch 400, Pérdida del predictor: 0.0005
Campaña 202404, Epoch 450, Pérdida del predictor: 0.0003
Campaña 202404, Epoch 500, Pérdida del predictor: 0.0003
Campaña 202404, Epoch 550, Pérdida del predictor: 0.0002
Campaña 202404, Epoch 600, Pérdida del predictor: 0.0002
Campaña 202404, Epoch 650, Pérdida del predictor: 0.0001
Campaña 202404, Epoch 700, Pérdida del predictor: 0.0001
Campaña 202404, Epoch 750, Pérdida del predictor: 0.0001
Campaña 202404, Epoch 800, P

c:\Users\sheil\OneDrive\Desktop\Proyecto_Tesis\tesis\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\sheil\OneDrive\Desktop\Proyecto_Tesis\tesis\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Aristas positivas guardadas en output\CY\aristas_positivas_campaña_202404.json
Grafo con aristas positivas guardado en output\CY\grafo_aristas_positivas_campaña_202404.png
Resultados guardados en output\CY\resultados_por_campaña.json
